In [ ]:
import sys
sys.path.append("src")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display

from implied_vol import implied_volatility
from bs import black_scholes_price


In [ ]:
ticker_dropdown = widgets.Dropdown(
    options=['AAPL', 'MSFT', 'TSLA', 'GOOG', 'AMZN'],
    value='AAPL',
    description='Ticker:'
)
display(ticker_dropdown)


In [ ]:
def fetch_option_chain(ticker):
    stock = yf.Ticker(ticker)
    expiries = stock.options
    if not expiries:
        raise ValueError(f"No options available for {ticker}")
    expiry = expiries[0]  # nearest expiry for now
    opt_chain = stock.option_chain(expiry)
    calls = opt_chain.calls
    puts = opt_chain.puts
    return expiry, calls, puts

expiry, calls, puts = fetch_option_chain(ticker_dropdown.value)
calls.head()


In [ ]:
S0 = yf.Ticker(ticker_dropdown.value).history(period="1d")["Close"].iloc[-1]
r = 0.05   # risk-free rate assumption
T = 0.05   # placeholder, will compute from expiry

expiry_date = pd.to_datetime(expiry).date()
today = pd.Timestamp.today().date()
T = (expiry_date - today).days / 365.0

calls["implied_vol"] = calls.apply(
    lambda row: implied_volatility(
        market_price=row["lastPrice"],
        S=S0, K=row["strike"], T=T, r=r, option_type="call"
    ),
    axis=1
)

calls = calls.dropna(subset=["implied_vol"])
calls.head()


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(calls["strike"], calls["implied_vol"], marker="o")
plt.xlabel("Strike")
plt.ylabel("Implied Volatility")
plt.title(f"IV Smile for {ticker_dropdown.value}, Expiry {expiry}")
plt.grid(True)
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

all_data = []
stock = yf.Ticker(ticker_dropdown.value)

for expiry in stock.options[:5]:  # first 5 expiries
    expiry_date = pd.to_datetime(expiry).date()
    T = (expiry_date - today).days / 365.0
    calls = stock.option_chain(expiry).calls
    calls["implied_vol"] = calls.apply(
        lambda row: implied_volatility(
            market_price=row["lastPrice"],
            S=S0, K=row["strike"], T=T, r=r, option_type="call"
        ),
        axis=1
    )
    calls = calls.dropna(subset=["implied_vol"])
    for _, row in calls.iterrows():
        all_data.append([row["strike"], T, row["implied_vol"]])

df_surface = pd.DataFrame(all_data, columns=["strike","maturity","iv"])


In [ ]:
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_surface["strike"], df_surface["maturity"], df_surface["iv"], c=df_surface["iv"], cmap="viridis")
ax.set_xlabel("Strike")
ax.set_ylabel("Maturity (years)")
ax.set_zlabel("Implied Volatility")
ax.set_title(f"IV Surface for {ticker_dropdown.value}")
plt.show()
